In [1]:
import os
#from eztrack.fragility.sourcesink import lds_raw_sourcesink
from pathlib import Path

from mne_bids import BIDSPath, read_raw_bids

#from eztrack import (read_raw_eztrack, write_eztrack_bids,
#                     plot_result_heatmap)
#from episcalp.sourcesink.run_sourcesink_analysis import run_sourcesink_analysis, run_post_analysis
from episcalp.sourcesink.run_post_sourcesink_analysis import run_post_analysis

import matplotlib.pyplot as plt

In [2]:
bids_root = Path("D:/OneDriveParent/OneDrive - Johns Hopkins/Shared Documents/40Hz-30")
subjects = [f.name.replace("sub-", "") for f in bids_root.glob("sub-*") if os.path.isdir(f)]
session = 'initialvisit'
task='monitor'
run='01'
datatype='eeg'

channels_drop = ['C3', 'C4', 'FZ', 'CZ', 'PZ']

reference = 'monopolar'
winsize = 100
stepsize = 100
hfreq = 40
filtrange = f"0.5to{hfreq}Hz"
thresh = 30

sfreq = 200.0
method="sourcesink"

filt_chan = "None"

ext = ".vhdr"
fig_ext = ".png"

post_analysis = True
features = ["entropy", "variance", "skew", "kurtosis", "kldiv"]


deriv_root = bids_root / "derivatives"
figures_root = deriv_root / "figures"
deriv_chain = Path(method) / f"filt-{filtrange}" / f"thresh-{thresh}" / f"winsize-{winsize}" / f"stepsize-{stepsize}" / f"sfreq-{sfreq}Hz" / f"filt-{filt_chan}" / f"ref-{reference}"

ss_model_params = {
    'winsize': winsize,
    'stepsize': stepsize
}

In [3]:
def standardize_channels(channel_names):
    standard_montage_channels = ['Fp1', 'Fp2', 'F3', 'F4', 'P3', 'P4', 'C3', 'C4', 'O1', 'O2', 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'Fz', 'Cz', 'Pz']
    channel_map = dict()
    for ch in channel_names:
        for sch in standard_montage_channels:
            if ch.lower() == sch.lower():
                channel_map[ch] = sch
    return channel_map

In [6]:
for subject in subjects:
    print(f"Looking at: {subject}")
    deriv_path = deriv_root / deriv_chain / f"sub-{subject}"
    bids_path_ = BIDSPath(subject=subject, session=session, task=task, run=run, datatype=datatype, extension=ext, root=bids_root)
    bids_path = bids_path_.match()
    if len(bids_path) == 0:
        bids_path_.update(session="awake")
        bids_path = bids_path_.match()
    bids_path = bids_path[0]
    print(bids_path)
    if post_analysis:
        run_post_analysis(deriv_path=deriv_root, subject=subject, features=features)
    else:
        run_sourcesink_analysis(
            bids_path,
            reference=reference,
            resample_freq=None,
            deriv_path=deriv_root,
            figures_path=figures_root,
            extra_channels=channels_drop,
            overwrite=True
        )

Looking at: 011
D:\OneDriveParent\OneDrive - Johns Hopkins\Shared Documents\40Hz-30\sub-011\ses-initialvisit\eeg\sub-011_ses-initialvisit_task-monitor_run-01_eeg.vhdr
D:\OneDriveParent\OneDrive - Johns Hopkins\Shared Documents\40Hz-30\derivatives\sourcesink\monopolar\sub-011\sub-011_ses-initialvisit_task-monitor_run-01_desc-ssindmatrix_eeg.json
p: (14, 1), q: (14, 1)


--- Logging error ---
Traceback (most recent call last):
  File "c:\users\patri\appdata\local\programs\python\python38\lib\logging\handlers.py", line 70, in emit
    self.doRollover()
  File "c:\users\patri\appdata\local\programs\python\python38\lib\logging\handlers.py", line 171, in doRollover
    self.rotate(self.baseFilename, dfn)
  File "c:\users\patri\appdata\local\programs\python\python38\lib\logging\handlers.py", line 111, in rotate
    os.rename(source, dest)
PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'd:\\desktop\\eztrack\\.eztrack\\logging\\eztrack.log' -> 'd:\\desktop\\eztrack\\.eztrack\\logging\\eztrack.log.1'
Call stack:
  File "c:\users\patri\appdata\local\programs\python\python38\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\users\patri\appdata\local\programs\python\python38\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File

In [5]:
if == 0:
    for subject in subjects:
        print(f"Looking at: {subject}")
        deriv_path = deriv_root / deriv_chain / f"sub-{subject}"
        bids_path = BIDSPath(subject=subject, session=session, task=task, run=run, datatype=datatype, extension=ext, root=bids_root)
        try:
            raw = read_raw_bids(bids_path)
        except FileNotFoundError:
            bids_path.update(session="awake")
            raw = read_raw_bids(bids_path)
        channels_drop_sub = [ch for ch in channels_drop if ch in raw.ch_names]
        raw.drop_channels(channels_drop_sub)
        channel_map = standardize_channels(raw.ch_names)
        raw.rename_channels(channel_map)
        raw.set_montage('standard_1020')
        ss_ind_deriv, state_deriv = lds_raw_sourcesink(raw, reference=reference,
                                                       return_all=True, **ss_model_params)
        deriv_bids_path = bids_path.copy().update(
            suffix='desc-sourcesink_ieeg',
            extension='.npy',
            check=False
        )
        deriv_fpath = deriv_path / deriv_bids_path.basename
        ss_ind_deriv.save(deriv_fpath, overwrite=True)
        ss_ind_deriv.info['model_params'] = dict(stepsize=10)
        #fig_basename = deriv_bids_path.copy().update(extension=fig_ext).basename
        fig_basename = 'sink_index.png'
        show_plot=True

        # save the plot
        figures_path = deriv_root / 'figures' / deriv_chain / f'sub-{subject}'
        figures_fpath = figures_path / fig_basename


        plot_result_heatmap(
            result=ss_ind_deriv, figures_path=figures_path, fig_basename=fig_basename, ext=fig_ext, show=show_plot, cmap='jet'
        )

        ss_ind_data = ss_ind_deriv.get_data()
        ss_chs = ss_ind_deriv.ch_names
        ss_ind_row_sum = ss_ind_data.sum(axis=1)
        ss_ind_pmf = ss_ind_row_sum / ss_ind_row_sum.sum()
        fig_basename = "distribution.png"
        figures_fpath = figures_path / fig_basename
        fig, ax = plt.subplots()
        ax.bar(ss_chs, ss_ind_pmf)
        plt.savefig(figures_fpath)

SyntaxError: invalid syntax (<ipython-input-5-c0bdeaabf667>, line 1)